# Synthesize /nanana/ sequence 
Train VTL to synthesize /nanana/ sequence by Anqi Xu (a.xu.17@ucl.ac.uk)
Velum constraints included; Tongue constraints included 1% resistance and 20% hard change link
- Python 3.6.3 on win32

About LibROSA package
- http://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf
- https://librosa.github.io/librosa/index.html

In [32]:
import os
import ctypes
import time
import random
import xml.etree.ElementTree as et
import numpy as np
import matplotlib.pyplot as plt
import librosa
import shutil

%matplotlib inline

In [15]:
# Load VTL 
VTL = ctypes.cdll.LoadLibrary('vtlapi-2.1b/VocalTractLabApi64.so')

In [24]:
# input
SPEAKER_FILE = 'JD2.speaker'
GESTURE_FILE = 'banane.ges'
TARGET = 'banane-orig.wav'

# output
WAV_FILE = 'output/nanana.wav'
FEEDBACK = 'feedback.txt'

In [25]:
# Initialize synthesis settings
speaker_file_name = ctypes.c_char_p(SPEAKER_FILE.encode())
gesture_file_name = ctypes.c_char_p(GESTURE_FILE.encode())
wav_file_name = ctypes.c_char_p(WAV_FILE.encode())
feedback_file_name = ctypes.c_char_p(FEEDBACK.encode())

In [18]:
# Range of vocal tract parameter
VTP_max_min = np.array([[0,1],[-6,-3.5],[-0.5,0],[-7,0],
                        [-1,1],[-2,4],[0,1],[-0.1,1],
                        [0,0],[-3,4],[-3,1],[1.5,5.5],
                        [-3,2.5],[-3,4],[-3,5],[-4,2],
                        [-6,0],[-1.4,1.4],[-1.4,1.4],[-1.4,1.4],
                        [-1.4,1.4],[-0.05,-0.05],[-0.05,-0.05],[-0.05,-0.05]])

## Test sound

In [22]:
import IPython
IPython.display.Audio(TARGET)

## Adding Constaints

Tongue and velum constraints
- Constraints1: Tongue parameters constraints: whenever the tongue blade paramters (TBX/TBY) were adjusted, those of tongue tip and tongue body were also modified by 20% with 1% resistance

In [6]:
def constrained_parameterT(ii, TBX, TBY, current_VTP, VTP_max_min, percent_change = 0.2, resist = 0.99):
    if ii == 9 or 11: #TCX and TTX changes according to TBX; TCY and TTY changes according to TBY
        change = TBX - current_VTP[13]
    else: #TCY and TTY changes according to TBY
        change = TBY - current_VTP[14]
    #TCX constraints
    constrained = current_VTP[ii]*resist+change*percent_change
    
    if constrained == max(VTP_max_min[ii]): #make sure it stays within the VTP range
        constrained_parameter = max(VTP_max_min[ii])
    elif constrained == min(VTP_max_min[ii]):
        constrained_parameter = min(VTP_max_min[ii])
    else:
        constrained_parameter = constrained
    return  constrained_parameter

- Constraints2:tongue body and velum constraints; every time the TCY parameter changes, velum opening changes by 20% with 1% resistance

In [7]:
def constrained_parameterVO(ii, TCY, current_VTP, VTP_max_min, percent_change = 0.2, resist = 0.99 ):
    change = TCY - current_VTP[10]
    constraint = current_VTP[ii]*resist+change*percent_change
    
    if constraint == max(VTP_max_min[ii]):
        constrained_parameter = max(VTP_max_min[ii])
    elif constraint == min(VTP_max_min[ii]):
        constrained_parameter = min(VTP_max_min[ii])
    else:
        constrained_parameter = constraint
    return  constrained_parameter

## Generate random parameter

In [8]:
def randomVTP(current_VTP):
    HX=random.uniform(0,1)# Generate random number from 0 to 1
    HY=random.uniform(-6,-3.5)
    JX=random.uniform(-0.5,0)
    JA=random.uniform(-7,0)
    LP=random.uniform(-1,1)
    LD=random.uniform(-2,4)
    VS=random.uniform(0,1)
    WC=0; #Not included in Prom-on et al., 2014, use netrual parameter instead
    TBX=random.uniform(-3,4)
    TBY=random.uniform(-3,5)
    TRX=random.uniform(-4,2)
    TRY=random.uniform(-6,0) 
    TS1=random.uniform(-1.4,1.4)
    TS2=random.uniform(-1.4,1.4)
    TS3=random.uniform(-1.4,1.4)
    TS4=random.uniform(-1.4,1.4)
    MA1= -0.05
    MA2= -0.05
    MA3= -0.05
    #Get VTP of tongue body (TCX, TCY), tongue tip (TTX,TTY) and velum opening (VO) by constraints
    TCX = constrained_parameterT(9, TBX, TBY, current_VTP, VTP_max_min)
    TCY = constrained_parameterT(10, TBX, TBY, current_VTP, VTP_max_min)
    TTX = constrained_parameterT(11, TBX, TBY, current_VTP, VTP_max_min)
    TTY = constrained_parameterT(12, TBX, TBY,current_VTP, VTP_max_min)
    VO = constrained_parameterVO(7, TCY, current_VTP, VTP_max_min)
    random_params = np.array([HX,HY,JX,JA,LP,LD,VS,VO,WC,TCX,TCY,TTX,TTY,TBX,TBY,TRX,TRY,TS1,TS2,TS3,TS4,MA1,MA2,MA3])
    return  random_params

## MFCC function
The function takes sound file name as input and returns MFCC Matrix
<img src="https://www.researchgate.net/profile/Alexandra_Trujillo_J/publication/322578191/figure/fig1/AS:584167817498625@1516287869835/Block-diagram-of-the-MFCC-algorithm-recognize-speechcom-feature-extraction-mfcc.ppm">

This function use LibROSA package to call a MFCC function

In [9]:
def get_MFCC(sound_file_name):
    y, sr = librosa.load(sound_file_name, sr= 16000) #resample to 16000 Hz
    #details of MFCC
    n_mfcc = 40
    n_mels = 40
    n_fft = 512
    win_length = int(0.025*sr) #window time is 0.025
    hop_length = int(0.010*sr) #hop time is 0.01 
    window = 'hamming'
    fmin = 0
    fmax = 8000
    # Short time fourier transfrom setting
    stft_setting = librosa.stft(y, window=window, n_fft=n_fft, win_length=win_length, hop_length=hop_length)
    S = librosa.feature.melspectrogram(S=stft_setting, y=y, n_mels=n_mels, fmin=fmin, fmax=fmax)
    MFCC = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=n_mfcc)
    return MFCC

## Update speaker file function
This function takes two sets of vocal tract parameters (numpy arrays)

In [10]:
def update_speaker_file(params_1, params_2):
    tree = et.parse(SPEAKER_FILE)
    for i in range(23):
    #use getchildren to find the node that stores vocal tract parameter
        # /a/ parameter in the speaker file
        tree.getroot().getchildren()[0].getchildren()[1].getchildren()[0].getchildren()[i].set('value', str(params_1[i]))
        # /n/ parameter in the speaker file
        tree.getroot().getchildren()[0].getchildren()[1].getchildren()[40].getchildren()[i].set('value', str(params_2[i]))
    tree.write(SPEAKER_FILE)

## Fix header for non Windows OS

In [36]:
WAV_HEADER = (b'RIFF\x8c\x87\x00\x00WAVEfmt\x20\x10\x00\x00\x00\x01\x00\x01'
                      + b'\x00"V\x00\x00D\xac\x00\x00\x02\x00\x10\x00data')
def fix_header():
    wav_file = wav_file_name.value.decode()
    with open(wav_file, 'rb') as file_:
        content = file_.read()

    shutil.move(wav_file, wav_file + '.bkup')

    with open(wav_file, 'wb') as newfile:
        newcontent = WAV_HEADER + content[68:]
        newfile.write(newcontent)

    os.remove(wav_file + '.bkup')

#     print('Fixed header in %s.' % wav_file)

In [38]:
# run this once to fix the target file
with open(TARGET, 'rb') as file_:
    content = file_.read()

shutil.move(TARGET, TARGET + '.bkup')

with open(TARGET, 'wb') as newfile:
    newcontent = WAV_HEADER + content[68:]
    newfile.write(newcontent)

    os.remove(TARGET + '.bkup')

    print('Fixed header in %s.' % TARGET)

Fixed header in banane-orig.wav.


## Training function
The optimization is done by comparing the error of MFCC between the target and synthetic audio.

Every time there is an improvement in MFCC, the set of VTP will be adopted and recorded.

In every iteration, the VTP is random except for tongue tip (TTX/TTY), tongue body (TCX,TCY) and velum opening (V0), which are constrained by certain rules (see the tongue and velum constraints function).

Loss function using RMSE 

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/05/rmse.png" width=300px>

In [30]:
def training_session(iteration = 100000): 
    #get target mfcc
    target_mfcc =  get_MFCC(TARGET) 
    #the mfcc difference between the netrual sequence and target sequence
    current_sum_of_squares =  562844.7591388852
    #start from vocal tract parameters of schwas
    # VTP of schwa
    current_VTP_1=np.array([1.0,  -4.75,  0.0,   -2.0,
                            -0.07,  0.95,   0.0,   -0.1,
                            0.0,   -0.4,   -1.46,    3.5,
                            -1.0,    2.0,    0.5,     0.0,
                            0.0,    0.0,   0.06,    0.15,
                            0.15,   -0.05,  -0.05,  -0.05])
    # VTP of schwa
    current_VTP_2=np.array([1.0,  -4.75,  0.0,   -2.0,
                            -0.07,  0.95,   0.0,   -0.1,
                            0.0,   -0.4,   -1.46,    3.5,
                            -1.0,    2.0,    0.5,     0.0,
                            0.0,    0.0,   0.06,    0.15,
                            0.15,   -0.05,  -0.05,  -0.05])
    
    # VTP of schwa
    params_1 = np.array([1.0,  -4.75,  0.0,   -2.0,
                         -0.07,  0.95,   0.0,   -0.1,
                         0.0,   -0.4,   -1.46,    3.5,
                         -1.0,    2.0,    0.5,     0.0,
                         0.0,    0.0,   0.06,    0.15,
                         0.15,   -0.05,  -0.05,  -0.05])
    # VTP of schwa
    params_2 =np.array([1.0,  -4.75,  0.0,   -2.0,
                        -0.07,  0.95,   0.0,   -0.1,
                        0.0,   -0.4,   -1.46,    3.5,
                        -1.0,    2.0,    0.5,     0.0,
                        0.0,    0.0,   0.06,    0.15,
                        0.15,   -0.05,  -0.05,  -0.05])
    
    # empty array for taking a record of every sum of squares
    # the first value in the array is the initial ssq
    Every_SSQ = np.zeros(shape=[iteration+1, 1])
    Every_SSQ[0,:] = current_sum_of_squares 
    
    # empty array for taking a record of every set of VTP
    # the first value in the array is VTP of a schwa
    Every_VTP_1 = np.zeros(shape=[iteration+1, 24])
    Every_VTP_1[0,:] = params_1
    
    Every_VTP_2 = np.zeros(shape=[iteration+1, 24])
    Every_VTP_2[0,:] = params_2
    
    counter = 0;
    for i in range(iteration):
        counter += 1
        
        # get a random set of vocal tract parameters with tongue constraints
        params_1=randomVTP(current_VTP_1)
        params_2=randomVTP(current_VTP_2)
        
        update_speaker_file(params_1, params_2)
        
        # synthesis by VTL
        VTL.vtlGesToWav(speaker_file_name, gesture_file_name, wav_file_name, feedback_file_name)
        fix_header()
        #VTL.vtlClose()
        
        # caculate the MFCC of the synthesized audio
        mfcc = get_MFCC(WAV_FILE)
        #caculate the sum of squares
        sum_of_squares = np.sum((mfcc - target_mfcc)**2)
        
        # Take a record of the SSQ of every synthetic sequence
        Every_SSQ[counter,:] = sum_of_squares
        
        # Take a record of every set of VTP
        Every_VTP_1[counter,:] = params_1 
        Every_VTP_2[counter,:] = params_2
        
        if sum_of_squares < current_sum_of_squares:
            current_sum_of_squares = sum_of_squares
            current_VTP_1=params_1
            current_VTP_2=params_2
            os.rename('nanana.wav','nanana%i.wav'%counter)
            np.savetxt("Every_SSQ.csv", Every_SSQ, delimiter=",") #every sum of squares
            np.savetxt("Every_VTP_1.csv", Every_VTP_1, delimiter=",")
            np.savetxt("Every_VTP_2.csv", Every_VTP_2, delimiter=",")
        #report the progress every 1000 iterations
        if (counter % 1000==0):
            print('Progress report')
            print('Current_sum_of_sqaures: %i' % current_sum_of_squares)
            np.savetxt("Every_SSQ.csv", Every_SSQ, delimiter=",") #every sum of squares
            np.savetxt("Every_VTP_1.csv", Every_VTP_1, delimiter=",")
            np.savetxt("Every_VTP_2.csv", Every_VTP_2, delimiter=",")
    return Every_SSQ, Every_VTP_1, Every_VTP_2

## Train session

In [ ]:
start = time.time()
Every_SSQ, Every_VTP_1, Every_VTP_2 = training_session(iteration = 100000)
end = time.time()
time = end-start
print('Time: %i' % time)

/home/thanatlapthawan/anaconda3/lib/python3.6/site-packages/librosa/core/spectrum.py:865: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '


In [21]:
# Save the output
np.savetxt("Every_SSQ.csv", Every_SSQ, delimiter=",")
np.savetxt("Every_VTP_1.csv", Every_VTP_1, delimiter=",")
np.savetxt("Every_VTP_2.csv", Every_VTP_2, delimiter=",")

NameError: name 'Every_SSQ' is not defined

## Test Synthesis sound

In [28]:
IPython.display.Audio(WAV_FILE)